In [49]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('data/ckd_clean.csv')
df.head()

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,...,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,pedal_edema,anemia,class
0,47,80.0,1.020,1,0,0,0,0,0,121.000000,...,44,7800,5,1,1,0,0,0,0,1
1,7,50.0,1.020,4,0,0,0,0,0,103.333333,...,37,6000,5,0,0,0,0,0,0,1
2,62,80.0,1.010,2,3,0,0,0,0,423.000000,...,31,7500,2,0,1,0,1,0,1,1
3,47,70.0,1.005,4,0,0,1,1,0,117.000000,...,32,6700,3,1,0,0,1,1,1,1
4,51,80.0,1.010,2,0,0,0,0,0,106.000000,...,34,7299,4,0,0,0,0,0,0,1


In [119]:
X = df.drop(["class"], axis=1).values
y = df["class"].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((320, 24), (80, 24), (320, 1), (80, 1))

In [120]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

ACTIVATIONS = {
    "sigmoid": (sigmoid, sigmoid_derivative),
    "relu": (relu, relu_derivative),
}

def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8
    return -np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))

def binary_cross_entropy_derivative(y_true, y_pred):
    epsilon = 1e-8
    return (y_pred - y_true) / ((y_pred * (1 - y_pred)) + epsilon)

def l2_penalty(weights, alpha=0.001):
    return alpha * sum(np.sum(w ** 2) for w in weights)

class MLPBinaryClassifier:
    def __init__(self, input_size, hidden_layers, activations, learning_rate=0.01, epochs=1000):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.activations = activations
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = []
        self.biases = []
        self.activation_funcs = []
        self.activation_derivatives = []
        self._initialize_network()

    def _initialize_network(self):
        layer_sizes = [self.input_size] + self.hidden_layers + [1]
        
        if len(self.activations) != len(layer_sizes) - 1:
            raise ValueError("# activation functions != # layers")

        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01)
            self.biases.append(np.zeros((1, layer_sizes[i + 1])))

            act_fn, act_deriv = ACTIVATIONS[self.activations[i]]
            self.activation_funcs.append(act_fn)
            self.activation_derivatives.append(act_deriv)

    def forward(self, X):
        activations = [X]
        A = X
        for i in range(len(self.weights)):
            Z = np.dot(A, self.weights[i]) + self.biases[i]
            A = self.activation_funcs[i](Z)
            activations.append(A)
        return activations

    def backward(self, activations, y):
        grads_w = []
        grads_b = []
        
        delta = binary_cross_entropy_derivative(y, activations[-1]) * self.activation_derivatives[-1](activations[-1])

        for i in reversed(range(len(self.weights))):
            grads_w.insert(0, np.dot(activations[i].T, delta))
            grads_b.insert(0, np.sum(delta, axis=0, keepdims=True))
            if i > 0:
                delta = np.dot(delta, self.weights[i].T) * self.activation_derivatives[i - 1](activations[i])
        return grads_w, grads_b

    def update_weights(self, grads_w, grads_b):
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * grads_w[i]
            self.biases[i] -= self.learning_rate * grads_b[i]

    def train(self, X, y):
        for epoch in range(self.epochs):
            activations = self.forward(X)
            grads_w, grads_b = self.backward(activations, y)
            self.update_weights(grads_w, grads_b)
            if epoch % 100 == 0:
                loss = binary_cross_entropy(y, activations[-1]) + l2_penalty(self.weights, alpha=0.001)
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        return (self.forward(X)[-1] > 0.5).astype(int)

In [137]:
def test_data(model=model, df_input=pd.DataFrame(), X_input_test=None, y_input_test=None):
    if not(df_input.empty):
        X_input_test = df_input.drop('class', axis=1)
        y_input_test = df_input['class'].to_numpy()
    predictions_new = model.predict(X_input_test)
    accuracy_new = np.mean(predictions_new == y_input_test)
    print(f"Accuracy: {accuracy_new:.4f}")
    unique_values, counts = np.unique(predictions_new, return_counts=True)
    print("Unique Values: ", unique_values, "\tCounts: ", counts)
    return accuracy_new

In [130]:
model = MLPBinaryClassifier(
    input_size=24,
    hidden_layers=[8],
    activations=["relu", "sigmoid"],
    learning_rate=0.01,
    epochs=1000
)

model.train(X_train, y_train)
accuracy = test_data(X_input_test=X_test, y_input_test=y_test)

Epoch 0, Loss: 0.6931
Epoch 100, Loss: 0.0257
Epoch 200, Loss: 0.0297
Epoch 300, Loss: 0.0323
Epoch 400, Loss: 0.0342
Epoch 500, Loss: 0.0357
Epoch 600, Loss: 0.0369
Epoch 700, Loss: 0.0380
Epoch 800, Loss: 0.0389
Epoch 900, Loss: 0.0397
Accuracy: 1.0000
Unique Values:  [0 1] 	Counts:  [28 52]


## Testing New Data

In [131]:
df_new = pd.read_csv('data/ckd_new.csv')
df_new.head()

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,pedal_edema,anemia,class
0,3,72.530698,1.019955,0,0,0,0,0,0,107.849211,10,0.617017,141,3.758309,12,36,7647,4,0,0,0,0,0,0,1
1,10,64.589968,1.009816,0,0,0,0,0,0,140.894137,22,3.034662,137,3.785582,12,34,12310,4,0,0,0,0,0,0,1
2,10,69.851171,1.009070,4,0,1,1,0,1,103.989692,83,1.584797,137,5.044046,9,30,16193,4,0,0,0,1,0,0,1
3,9,115.312509,1.009427,3,0,0,0,0,0,121.522842,16,1.200639,137,4.498946,14,44,8943,4,0,0,0,0,0,0,1
4,18,59.925209,1.016364,0,0,0,0,0,0,165.192004,41,3.276387,137,3.235189,14,40,9386,5,0,1,0,1,1,0,1


In [132]:
accuracy_new_test = test_data(df_input=df_new)

Accuracy: 0.6400
Unique Values:  [1] 	Counts:  [200]


In [133]:
def round_to_nearest_values(df, column, valid_values):
    valid_values = np.array(valid_values)
    original_dtype = df[column].dtype

    def rounder(x):
        return valid_values[np.argmin(np.abs(valid_values - x))]

    df[column] = df[column].apply(rounder).astype(original_dtype)
    return df

In [134]:
df_new_test = df_new.copy()
df_new_test = round_to_nearest_values(df_new_test, 'age', df['age'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'blood_pressure', df['blood_pressure'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'specific_gravity', df['specific_gravity'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'albumin', df['albumin'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'sugar', df['sugar'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'blood_glucose_random', df['blood_glucose_random'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'blood_urea', df['blood_urea'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'serum_creatinine', df['serum_creatinine'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'sodium', df['sodium'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'potassium', df['potassium'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'haemoglobin', df['haemoglobin'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'packed_cell_volume', df['packed_cell_volume'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'white_blood_cell_count', df['white_blood_cell_count'].unique())
df_new_test = round_to_nearest_values(df_new_test, 'red_blood_cell_count', df['red_blood_cell_count'].unique())
df_new_test.head()

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,pedal_edema,anemia,class
0,3,73.333333,1.020000,0,0,0,0,0,0,108.000000,10,0.6,141,3.766667,12,36,7600,4,0,0,0,0,0,0,1
1,11,63.333333,1.010000,0,0,0,0,0,0,141.000000,22,3.0,137,3.800000,12,34,12300,4,0,0,0,0,0,0,1
2,11,70.000000,1.008333,4,0,1,1,0,1,104.000000,83,1.6,137,5.000000,9,30,16300,4,0,0,0,1,0,0,1
3,8,120.000000,1.010000,3,0,0,0,0,0,121.666667,16,1.2,137,4.500000,14,44,8966,4,0,0,0,0,0,0,1
4,17,60.000000,1.016667,0,0,0,0,0,0,165.000000,41,3.3,137,3.200000,14,40,9400,5,0,1,0,1,1,0,1


In [135]:
accuracy_new_test = test_data(df_input=df_new_test)

Accuracy: 0.6400
Unique Values:  [1] 	Counts:  [200]


## Training and Testing Original + New Data Combined

In [138]:
df_combined = pd.concat([df, df_new], ignore_index=True)
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)
df_combined.head()

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,pedal_edema,anemia,class
0,63,90.000000,1.015000,0,0,0,0,0,0,123.000000,19,2.000000,142,3.800000,11,34,11400,4,0,0,0,0,0,0,1
1,33,99.764964,1.020829,0,0,0,0,0,0,140.506692,39,2.099372,145,5.195538,14,43,9189,5,0,0,0,0,0,0,0
2,69,92.726478,1.019185,2,0,1,0,0,0,145.935496,98,4.312420,132,5.206486,11,40,7530,4,1,0,0,0,0,0,1
3,67,70.000000,1.010000,1,0,0,0,0,0,102.000000,48,3.200000,137,5.000000,11,34,7100,3,1,1,0,0,1,0,1
4,45,70.000000,1.025000,2,0,0,1,1,0,117.000000,52,2.200000,136,3.800000,10,30,19100,3,0,0,0,0,0,0,1


In [140]:
X_combined = df_combined.drop(["class"], axis=1).values
y_combined = df_combined["class"].values.reshape(-1, 1)

X_combined_train, X_combined_test, y_combined_train, y_combined_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_combined_train = scaler.fit_transform(X_combined_train)
X_combined_test = scaler.transform(X_combined_test)

print(X_combined_train.shape, X_combined_test.shape, y_combined_train.shape, y_combined_test.shape)

model_combined = MLPBinaryClassifier(
    input_size=24,
    hidden_layers=[8],
    activations=["relu", "sigmoid"],
    learning_rate=0.01,
    epochs=1000
)

model_combined.train(X_combined_train, y_combined_train)
accuracy = test_data(
    model=model_combined, 
    X_input_test=X_combined_test, 
    y_input_test=y_combined_test
)

(480, 24) (120, 24) (480, 1) (120, 1)
Epoch 0, Loss: 0.6933
Epoch 100, Loss: 0.0359
Epoch 200, Loss: 0.0432
Epoch 300, Loss: 0.0484
Epoch 400, Loss: 0.0523
Epoch 500, Loss: 0.0555
Epoch 600, Loss: 0.0580
Epoch 700, Loss: 0.0602
Epoch 800, Loss: 0.0621
Epoch 900, Loss: 0.0638
Accuracy: 1.0000
Unique Values:  [0 1] 	Counts:  [45 75]
